<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
!pip install pandas==2.2.2 -qq
!pip install pyxirr -qq
!pip install ta -qq
!pip install yfinance==0.2.59 -qq
'''

'\n!pip install pandas==2.2.2 -qq\n!pip install pyxirr -qq\n!pip install ta -qq\n!pip install yfinance==0.2.59 -qq\n'

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  #stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-02-18 13:11:08
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.33 C
CY Investment:  1.60 C
Reserve:  4.98 L
Current:  1.43 C
-------------------
Today PnL: -13.83 K (-0.1%)
Current PnL: -31.50 L (-19.69%)
CY Booked + Current PnL: -15.82 L (-9.89%)
-------------------
Total profit:  1.30 L
Total loss:  -32.79 L
-------------------
Total Booked + Current PnL: 11.16 L (8.39%)
Total Booked PnL: 42.66 L (32.07%)
Curr Year Booked PnL: 15.68 L (10.97%)
Prev Year Booked PnL: 26.98 L (20.29%)
Est FTT:  2.40 C
Est FTT PnL: 96.84 L (67.75%)
Deployed:  1.33 C
Current:  1.43 C
CAGR/XIRR %: 3.74%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,MASFIN,0.18,3.61,17.33,21.57,17593.0,3540.0,101520.0,397.04,-10.57,60.0,H-SC,4.90,167.0,0.20,0.74,48.15,XR,ATH,FINANCE
18,COALINDIA,-0.16,2.15,15.19,17.67,23347.0,3238.0,153698.0,478.51,23.39,49.0,L-LC,11.33,185.0,0.14,1.11,23.38,XY25,ATH,MINING
77,TTKPRESTIG,-0.60,-27.75,38.53,0.09,28056.0,-27961.0,72816.0,770.00,69.05,38.0,M-SC,9.38,250.0,-1.00,0.53,0.22,OX40N,NTT,DURABLES
36,ICICIGI,1.00,3.16,15.68,19.34,32369.0,6326.0,206435.0,2252.93,-13.21,63.0,X-MC,2.85,66.0,0.20,1.50,19.60,X40,ATH,INSURANCE
51,MEDANTA,0.41,-1.71,26.23,24.08,32731.0,-2167.0,124783.0,1486.00,-2.35,58.0,X-SC,11.00,87.0,-0.07,0.90,14.29,XY24,NTT,HEALTHCARE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,EASEMYTRIP,9.58,-42.55,153.60,45.70,156028.0,-75234.0,101581.0,26.4,5.52,85.0,M-SC,22.53,204.0,-0.48,0.74,67.36,XY24,NTT,TRAVEL
62,SATIN,2.46,-11.91,72.36,51.83,176112.0,-32901.0,243383.0,274.0,-21.16,60.0,H-SC,4.31,159.0,-0.19,1.76,18.56,XY24,NTT,FINANCE
44,ITC,1.89,-18.73,36.31,10.78,70918.0,-45006.0,195312.0,452.0,-51.88,55.0,X-LC,18.66,1.0,-0.63,1.42,7.58,X40,NTT,FMCG
49,LAOPALA,1.79,-41.32,137.01,39.09,105413.0,-54167.0,76938.0,464.0,68.37,46.0,H-SC,15.32,154.0,-0.51,0.56,2.89,AR,NTT,CERAMICS
78,UNITDSPR,1.64,4.61,15.45,20.77,37841.0,10784.0,244928.0,1644.0,-7.24,64.0,X-MC,1.93,64.0,0.28,1.78,12.35,X40N,NTT,BREWERIES


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BSOFT,-2.81,-32.01,121.79,50.79,116462.0,-45028.0,95625.0,831.70,-12.64,36.0,H-SC,7.00,172.0,-0.39,0.69,11.34,XR,ATH,IT
39,INDIAMART,-2.66,-6.98,122.24,106.72,140239.0,-8612.0,114724.0,4810.62,-58.01,42.0,H-SC,7.82,140.0,-0.06,0.83,14.76,AR,ATH,MISC
84,WIPRO,-2.33,-13.60,99.37,72.27,145487.0,-23039.0,146409.0,420.00,-22.21,26.0,M-LC,6.46,98.0,-0.16,1.06,0.00,XR,NTT,IT
79,VAIBHAVGBL,-1.90,-23.95,111.40,60.77,154844.0,-43777.0,138998.0,521.00,60.33,55.0,H-SC,1.46,160.0,-0.28,1.01,26.02,XR,NTT,JEWELLERY
0,5PAISA,-1.88,-38.83,84.56,12.90,113024.0,-84835.0,133661.0,593.00,97.12,40.0,H-SC,6.19,170.0,-0.75,0.97,11.14,OX40N,NTT,FINANCE


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,FINCABLES,-0.20,3.48,100.91,107.91,154178.0,5144.0,152788.0,1641.55,-7.15,64.0,M-SC,8.81,220.0,0.03,1.11,14.95,OX40N,ATH,CABLES
77,TTKPRESTIG,-0.60,-27.75,38.53,0.09,28056.0,-27961.0,72816.0,770.00,69.05,38.0,M-SC,9.38,250.0,-1.00,0.53,0.22,OX40N,NTT,DURABLES
66,SIS,0.73,-28.35,69.23,21.26,55079.0,-31472.0,79560.0,528.00,1871.50,37.0,H-SC,4.23,168.0,-0.57,0.58,9.68,OX40N,NTT,MISC
72,TATAELXSI,-0.96,-29.78,88.03,32.03,81488.0,-39267.0,92568.0,9161.00,-12.13,35.0,H-SC,7.24,158.0,-0.48,0.67,3.68,OX40N,NTT,IT
40,INDIGOPNTS,-0.31,-30.74,44.41,0.02,53692.0,-53659.0,120900.0,1408.00,84.15,30.0,M-SC,8.68,222.0,-1.00,0.88,5.16,OX40N,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VOLTAS,0.3,20.03,20.97,45.2,48261.0,38403.0,230145.0,1856.0,37.4,67.0,X-MC,4.47,79.0,0.8,1.67,28.19,XY25,NTT,AC


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COALINDIA,-0.16,2.15,15.19,17.67,23347.0,3238.0,153698.0,478.51,23.39,49.0,L-LC,11.33,185.0,0.14,1.11,23.38,XY25,ATH,MINING
26,FINCABLES,-0.20,3.48,100.91,107.91,154178.0,5144.0,152788.0,1641.55,-7.15,64.0,M-SC,8.81,220.0,0.03,1.11,14.95,OX40N,ATH,CABLES
50,MASFIN,0.18,3.61,17.33,21.57,17593.0,3540.0,101520.0,397.04,-10.57,60.0,H-SC,4.90,167.0,0.20,0.74,48.15,XR,ATH,FINANCE
1,ABB,-0.03,7.04,36.00,45.57,100812.0,18413.0,280032.0,7934.00,-31.20,68.0,H-MC,4.30,125.0,0.18,2.03,24.35,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,MASFIN,0.18,3.61,17.33,21.57,17593.0,3540.0,101520.0,397.04,-10.57,60.0,H-SC,4.90,167.0,0.20,0.74,48.15,XR,ATH,FINANCE
1,ABB,-0.03,7.04,36.00,45.57,100812.0,18413.0,280032.0,7934.00,-31.20,68.0,H-MC,4.30,125.0,0.18,2.03,24.35,AR,NTT,ELECTRICAL
26,FINCABLES,-0.20,3.48,100.91,107.91,154178.0,5144.0,152788.0,1641.55,-7.15,64.0,M-SC,8.81,220.0,0.03,1.11,14.95,OX40N,ATH,CABLES
39,INDIAMART,-2.66,-6.98,122.24,106.72,140239.0,-8612.0,114724.0,4810.62,-58.01,42.0,H-SC,7.82,140.0,-0.06,0.83,14.76,AR,ATH,MISC
85,ZYDUSLIFE,-0.01,-4.40,41.88,35.63,83906.0,-9226.0,200348.0,1286.17,-16.67,52.0,H-MC,5.43,120.0,-0.11,1.45,11.54,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
42,INFY,-1.65,-6.86,44.13,34.24,128607.0,-21461.0,291427.0,1972.00,-26.71,24.0,X-LC,3.94,3.0,-0.17,2.11,0.86,X40,NTT,IT
73,TCS,-1.08,-26.09,60.40,18.55,168849.0,-98685.0,279552.0,4311.59,-33.14,26.0,X-LC,4.22,2.0,-0.58,2.03,0.00,X40,ATH,IT
33,HCLTECH,-1.28,-5.02,30.38,23.83,69809.0,-12139.0,229785.0,1908.19,-0.33,27.0,X-LC,4.54,8.0,-0.17,1.67,10.25,X40,ATH,IT
8,AWL,-0.65,-34.30,139.25,57.19,298378.0,-111852.0,214275.0,485.00,-62.47,30.0,X-SC,17.56,81.0,-0.37,1.55,0.57,XY24,NTT,FMCG
13,BERGEPAINT,-0.57,-17.92,47.59,21.14,88805.0,-40735.0,186604.0,680.00,-19.66,30.0,X-MC,12.42,68.0,-0.46,1.35,0.49,XY24,NTT,PAINTS


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,DABUR,-1.63,-0.60,44.57,43.70,109898.0,-1494.0,246574.0,735.00,-9.86,47.0,X-MC,1.87,70.0,-0.01,1.79,14.36,XY24,BTT,FMCG
78,UNITDSPR,1.64,4.61,15.45,20.77,37841.0,10784.0,244928.0,1644.00,-7.24,64.0,X-MC,1.93,64.0,0.28,1.78,12.35,X40N,NTT,BREWERIES
65,SIEMENS,0.23,-14.19,46.28,25.51,73900.0,-26415.0,159680.0,4671.50,34.19,57.0,H-LC,1.95,51.0,-0.36,1.16,17.67,AR,ATH,ELECTRICAL
36,ICICIGI,1.00,3.16,15.68,19.34,32369.0,6326.0,206435.0,2252.93,-13.21,63.0,X-MC,2.85,66.0,0.20,1.50,19.60,X40,ATH,INSURANCE
34,HINDUNILVR,0.46,-9.63,25.79,13.67,59910.0,-24763.0,232300.0,2922.00,-32.95,43.0,X-LC,2.86,9.0,-0.41,1.68,9.67,XY25,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,-1.08,-26.09,60.40,18.55,168849.0,-98685.0,279552.0,4311.59,-33.14,26.0,X-LC,4.22,2.0,-0.58,2.03,0.00,X40,ATH,IT
53,PGHH,-0.10,-13.22,52.12,32.01,96576.0,-28224.0,185296.0,17616.87,-41.36,35.0,X-MC,9.32,57.0,-0.29,1.34,0.15,X40,ATH,FMCG
13,BERGEPAINT,-0.57,-17.92,47.59,21.14,88805.0,-40735.0,186604.0,680.00,-19.66,30.0,X-MC,12.42,68.0,-0.46,1.35,0.49,XY24,NTT,PAINTS
3,ACC,0.10,-31.04,138.03,64.15,226507.0,-73851.0,164100.0,3906.00,-48.05,40.0,X-SC,8.47,85.0,-0.33,1.19,0.54,XY24,BTT,CEMENT
8,AWL,-0.65,-34.30,139.25,57.19,298378.0,-111852.0,214275.0,485.00,-62.47,30.0,X-SC,17.56,81.0,-0.37,1.55,0.57,XY24,NTT,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,-0.03,-30.07,104.28,42.84,47402.0,-19550.0,45456.0,424.00,-54.27,52.0,X-SC,18.85,80.0,-0.41,0.33,9.34,XY24,NTT,MISC
12,BATAINDIA,-0.57,-46.20,151.30,35.20,104739.0,-59444.0,69226.0,2096.00,-9.17,36.0,X-SC,21.52,91.0,-0.57,0.50,0.63,X40,NTT,FOOTWEAR
58,RELAXO,0.39,-52.13,218.18,52.31,151602.0,-75675.0,69485.0,1176.00,-49.09,46.0,X-SC,14.06,92.0,-0.50,0.50,4.17,X40N,NTT,FOOTWEAR
51,MEDANTA,0.41,-1.71,26.23,24.08,32731.0,-2167.0,124783.0,1486.00,-2.35,58.0,X-SC,11.00,87.0,-0.07,0.90,14.29,XY24,NTT,HEALTHCARE
35,HONAUT,0.08,-22.88,85.85,43.32,107828.0,-37272.0,125600.0,58357.33,-32.25,39.0,X-SC,11.15,90.0,-0.35,0.91,1.98,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,ITC,1.89,-18.73,36.31,10.78,70918.0,-45006.0,195312.0,452.00,-51.88,55.0,X-LC,18.66,1.0,-0.63,1.42,7.58,X40,NTT,FMCG
73,TCS,-1.08,-26.09,60.40,18.55,168849.0,-98685.0,279552.0,4311.59,-33.14,26.0,X-LC,4.22,2.0,-0.58,2.03,0.00,X40,ATH,IT
42,INFY,-1.65,-6.86,44.13,34.24,128607.0,-21461.0,291427.0,1972.00,-26.71,24.0,X-LC,3.94,3.0,-0.17,2.11,0.86,X40,NTT,IT
75,TMPV,-0.56,-21.97,57.60,22.98,101528.0,-49634.0,176264.0,600.00,-82.53,62.0,X-LC,2.96,4.0,-0.49,1.28,12.68,XY24,NTT,AUTO
33,HCLTECH,-1.28,-5.02,30.38,23.83,69809.0,-12139.0,229785.0,1908.19,-0.33,27.0,X-LC,4.54,8.0,-0.17,1.67,10.25,X40,ATH,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INDUSINDBK,-0.22,-30.48,91.14,32.89,48921.0,-23530.0,53677.0,1800.00,-352.80,63.0,L-MC,11.11,258.0,-0.48,0.39,47.82,XR,NTT,BANKS
67,SONACOMS,-0.16,-8.50,52.02,39.09,48149.0,-8601.0,92558.0,804.02,-27.49,63.0,M-MC,6.67,193.0,-0.18,0.67,30.94,AR,ATH,AUTO
25,EASEMYTRIP,9.58,-42.55,153.60,45.70,156028.0,-75234.0,101581.0,26.40,5.52,85.0,M-SC,22.53,204.0,-0.48,0.74,67.36,XY24,NTT,TRAVEL
50,MASFIN,0.18,3.61,17.33,21.57,17593.0,3540.0,101520.0,397.04,-10.57,60.0,H-SC,4.90,167.0,0.20,0.74,48.15,XR,ATH,FINANCE
79,VAIBHAVGBL,-1.90,-23.95,111.40,60.77,154844.0,-43777.0,138998.0,521.00,60.33,55.0,H-SC,1.46,160.0,-0.28,1.01,26.02,XR,NTT,JEWELLERY


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,EASEMYTRIP,9.58,-42.55,153.60,45.70,156028.0,-75234.0,101581.0,26.40,5.52,85.0,M-SC,22.53,204.0,-0.48,0.74,67.36,XY24,NTT,TRAVEL
7,ATULAUTO,-0.17,-11.40,64.30,45.57,112305.0,-22474.0,174658.0,844.00,3786.49,68.0,M-SC,4.42,248.0,-0.20,1.27,32.18,XY24,NTT,AUTO
11,BANDHANBNK,0.30,-18.98,137.49,92.42,310309.0,-52863.0,225696.0,400.00,93.25,68.0,H-SC,7.48,174.0,-0.17,1.64,30.43,XY24,NTT,BANKS
1,ABB,-0.03,7.04,36.00,45.57,100812.0,18413.0,280032.0,7934.00,-31.20,68.0,H-MC,4.30,125.0,0.18,2.03,24.35,AR,NTT,ELECTRICAL
67,SONACOMS,-0.16,-8.50,52.02,39.09,48149.0,-8601.0,92558.0,804.02,-27.49,63.0,M-MC,6.67,193.0,-0.18,0.67,30.94,AR,ATH,AUTO


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.18
1,25,44.66
2,50,76.74


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    46.39
MC    30.35
LC    23.26
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.94
X40      23.18
X40N     15.19
AR        8.99
XY25      8.83
XR        8.70
OX40N     7.30
SR        0.87
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    22.89
H-SC    22.67
X-LC    19.93
X-SC    11.85
M-SC    11.09
H-MC     4.95
M-MC     2.12
H-LC     1.16
L-LC     1.11
M-LC     1.06
L-SC     0.78
L-MC     0.39
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.05,-14.04,52.36
FINANCE,13.45,-12.47,56.73
IT,11.43,-35.53,103.17
MISC,7.06,-33.36,87.91
ELECTRICAL,6.28,-8.34,47.87
PAINTS,5.09,-31.50,51.03
INSURANCE,4.90,-0.39,34.74
PHARMA,4.11,-3.67,36.30
AUTO,3.22,-18.20,59.07


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3252763.0,21
AR,1331189.0,10
X40,1298740.0,15
XR,1291146.0,12
X40N,1025787.0,10
OX40N,786494.0,10
XY25,398307.0,6
SR,299540.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3666040.0,24
M-SC,1389130.0,14
X-MC,1355668.0,15
X-SC,1167415.0,10
X-LC,1142495.0,12
H-MC,395233.0,3
L-SC,183076.0,2
M-LC,145487.0,1
M-MC,93254.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1237417.0      6
           AR         950391.0      5
           XR         830429.0      7
M-SC       XY24       793635.0      6
X-SC       XY24       605018.0      4
X-LC       X40        582383.0      6
X-MC       X40        540560.0      7
           X40N       413429.0      4
X-SC       X40N       386600.0      4
H-SC       OX40N      348263.0      4
M-SC       OX40N      344686.0      5
H-SC       SR         299540.0      2
X-LC       X40N       225758.0      2
H-MC       XY24       210515.0      1
X-LC       XY24       207475.0      2
X-MC       XY25       202976.0      2
           XY24       198703.0      2
H-MC       AR         184718.0      2
M-SC       XR         176778.0      2
X-SC       X40        175797.0      2
M-LC       XR         145487.0      1
X-LC       XY25       126879.0      2
L-SC       OX40N       93545.0      1
           XR          89531.0      1
M-SC       AR          74031.0      1
H-LC       AR          73900.0      1
L-MC       XR          48921.0      1
M-MC       AR          48149.0      1
           XY25        45105.0      1
L-LC       XY25        23347.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 30.0 seconds
